In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,AlphaDropout,SeparableConv2D, SpatialDropout2D, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
#Loading Labels
traindf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/traindf.xlsx')
valdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/valdf.xlsx')
testdf = pd.read_excel('/content/drive/My Drive/Paper/UTKFace/testdf.xlsx')
traindf.head()

In [ ]:
traindf['gender'].value_counts()

0    9900
1    9061
Name: gender, dtype: int64

In [ ]:
#Loading Features Extracted using ResNet50

fx_train_res_pp= np.load('/content/drive/My Drive/Paper/UTKFace/fx_train_res_pp.npy')
fx_val_res_pp = np.load('/content/drive/My Drive/Paper/UTKFace/fx_val_res_pp.npy')
fx_test_res_pp = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_res_pp.npy')
print("Loaded!")

Loaded!


In [ ]:
x_train= np.load('/content/drive/My Drive/Paper/UTKFace/x_train.npy')
x_val = np.load('/content/drive/My Drive/Paper/UTKFace/x_val.npy')
x_test = np.load('/content/drive/My Drive/Paper/UTKFace/x_test.npy')

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git -q

In [ ]:
from keras_vggface.vggface import VGGFace
vgg16 = VGGFace(model='vgg16',include_top=False, input_shape=(200, 200, 3))

58916864/58909280 [==============================] - 1s 0us/step


In [ ]:
fx_train_vgg = vgg16.predict(x_train)
fx_val_vgg = vgg16.predict(x_val)
fx_test_vgg = vgg16.predict(x_test)

In [ ]:
np.save('/content/drive/My Drive/Paper/UTKFace/fx_train_vgg.npy',fx_train_vgg)
np.save('/content/drive/My Drive/Paper/UTKFace/fx_val_vgg.npy',fx_val_vgg)
np.save('/content/drive/My Drive/Paper/UTKFace/fx_test_vgg.npy',fx_test_vgg)

Gender Classification Model

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',np.unique(traindf['gender'].values),traindf['gender'].values)
class_weights

array([0.95762626, 1.04629732])

In [ ]:
print('Train : '+str(fx_train_vgg.shape))
print('Val : '+str(fx_val_vgg.shape))
print('Test : '+str(fx_test_vgg.shape))

In [ ]:
model = Sequential()

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.5))
model.add(SeparableConv2D(512,(3,3),padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.5))
model.add(SeparableConv2D(512,(3,3),padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
#model.add(BatchNormalization())
#model.add(AlphaDropout(0.5))
#model.add(Dense(1024,activation='relu',kernel_initializer='he_uniform'))

#model.add(BatchNormalization())
#model.add(AlphaDropout(0.5))
#model.add(Dense(512,activation='relu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(AlphaDropout(0.5))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))

In [ ]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=1e-4, momentum=0, nesterov=False, name="SGD")

adam = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    amsgrad=True,)

adamax = tf.keras.optimizers.Adamax(learning_rate=1e-4)

model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
!pip install keras-adabound -q

In [ ]:
model.fit(fx_train_vgg,traindf['gender'].values,epochs=10, validation_data=(fx_val_vgg,valdf['gender'].values),class_weight=class_weights,batch_size=64)

Train on 18961 samples, validate on 2370 samples
Epoch 1/10
18961/18961 [==============================] - 6s 327us/step - loss: 0.1772 - accuracy: 0.9270 - val_loss: 0.2969 - val_accuracy: 0.9295
Epoch 2/10
18961/18961 [==============================] - 6s 325us/step - loss: 0.1785 - accuracy: 0.9262 - val_loss: 0.2720 - val_accuracy: 0.9295
Epoch 3/10
18961/18961 [==============================] - 6s 316us/step - loss: 0.1768 - accuracy: 0.9264 - val_loss: 0.2747 - val_accuracy: 0.9278
Epoch 4/10
18961/18961 [==============================] - 6s 317us/step - loss: 0.1740 - accuracy: 0.9273 - val_loss: 0.2801 - val_accuracy: 0.9287
Epoch 5/10
18961/18961 [==============================] - 6s 317us/step - loss: 0.1712 - accuracy: 0.9299 - val_loss: 0.2742 - val_accuracy: 0.9278
Epoch 6/10
18961/18961 [==============================] - 6s 317us/step - loss: 0.1689 - accuracy: 0.9304 - val_loss: 0.2869 - val_accuracy: 0.9262
Epoch 7/10
18961/18961 [==============================] - 6s 31

In [ ]:
model.evaluate(fx_test_vgg,testdf['gender'].values)

2371/2371 [==============================] - 1s 214us/step


[0.28907131426993077, 0.9342049956321716]

In [ ]:
model.save('/content/drive/My Drive/Paper/UTKFace/Gender/VGG16/model.h5')

In [ ]:
model.predict(fx_test_vgg).shape

Ensemble

In [ ]:
from keras.models import load_model

G_res = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/ResNet50/9464.h5')
G_res.compile("Adam","binary_crossentropy",['acc'])

G_sen = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/SeNet50/9494.h5')
G_sen.compile("Adam","binary_crossentropy",['acc'])

G_vgg = load_model('/content/drive/My Drive/Paper/UTKFace/Gender/VGG16/9367.h5')
G_vgg.compile("Adam","binary_crossentropy",['acc'])

In [ ]:
fx_test_res_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_res.npy')
fx_test_sen_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_sen.npy')
fx_test_vgg_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_vgg.npy')

In [ ]:
pred_res = G_res.predict(fx_test_res_e)
pred_sen = G_sen.predict(fx_test_sen_e)
pred_vgg = G_vgg.predict(fx_test_vgg_e)

In [ ]:
pred_res.shape, pred_sen.shape, pred_vgg.shape

In [ ]:
pred_res[:5]

In [ ]:
pred_sen[:5]

In [ ]:
pred_vgg[:5]

In [ ]:
y_pred = (pred_res+pred_sen+pred_vgg)/3

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(testdf['gender'].values,y_pred.round())
print(report)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1234
           1       0.95      0.94      0.95      1137

    accuracy                           0.95      2371
   macro avg       0.95      0.95      0.95      2371
weighted avg       0.95      0.95      0.95      2371



In [ ]:
from sklearn.metrics import roc_auc_score

auc_vgg = roc_auc_score(testdf['gender'].values,pred_vgg.round())
auc_res = roc_auc_score(testdf['gender'].values,pred_res.round())
auc_sen = roc_auc_score(testdf['gender'].values,pred_sen.round())
auc_en = roc_auc_score(testdf['gender'].values,y_pred.round())
print('ROC_AUC_SCORE : \n')
print('ResNet50: ',auc_res)
print('SeNet50: ',auc_sen)
print('VGG16: ',auc_vgg)
print('Ensemble: ',auc_en)

ROC_AUC_SCORE : 

ResNet50:  0.9465709899376933
SeNet50:  0.9491653231726701
VGG16:  0.9361109804441441
Ensemble:  0.9477423599024417


In [ ]:
from sklearn.metrics import accuracy_score 

acc_vgg = accuracy_score(testdf['gender'].values,pred_vgg.round())
acc_res = accuracy_score(testdf['gender'].values,pred_res.round())
acc_sen = accuracy_score(testdf['gender'].values,pred_sen.round())
acc_en = accuracy_score(testdf['gender'].values,y_pred.round())
print('ACCURACY_SCORE : \n')
print('ResNet50: ',acc_res)
print('SeNet50: ',acc_sen)
print('VGG16: ',acc_vgg)
print('Ensemble: ',acc_en)

ACCURACY_SCORE : 

ResNet50:  0.9464361029101644
SeNet50:  0.9493884436946436
VGG16:  0.9367355546183045
Ensemble:  0.9481231547870097
